In [1]:
import sys, os
import pickle

import pandas as pd
from itertools import combinations

from github import Github

sys.path.append(os.path.abspath('package'))

from package.knowledge_graph import KnowledgeGraphBuilder

C:\Users\szmid\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open ('./_/git_token.txt', 'r') as f:
    git_token = f.read().strip()

kgb = KnowledgeGraphBuilder(git_token)

In [3]:
# Only create the graph to the feature extraction repository for faster workflow and visualization
repograph = kgb.build_knowledge_graph(
    repo_path='./sklearn/sklearn/', 
    repo_name='scikit-learn/scikit-learn',
    graph_type='CFG',
    num_of_PRs=120
)

Pulls done
Processing PR # 0
Processing PR # 1
Processing PR # 2
Processing PR # 3
Processing PR # 4
Processing PR # 5
Processing PR # 6
Processing PR # 7
Processing PR # 7
Processing PR # 8
Processing PR # 9
Processing PR # 10
Processing PR # 11
Processing PR # 12
Processing PR # 13
Processing PR # 14
Processing PR # 15
Processing PR # 16
Processing PR # 17
Processing PR # 18
Processing PR # 19
Processing PR # 20
Processing PR # 21
Processing PR # 22
Processing PR # 23
Processing PR # 24
Processing PR # 25
Processing PR # 26
Processing PR # 27
Processing PR # 28
Processing PR # 29
Processing PR # 30
Processing PR # 31
Processing PR # 32
Processing PR # 33
Processing PR # 34
Processing PR # 35
Processing PR # 36
Processing PR # 37
Processing PR # 38
Processing PR # 39
Processing PR # 40
Processing PR # 41
Processing PR # 42
Processing PR # 43
Processing PR # 44
Processing PR # 45
Processing PR # 46
Processing PR # 47
Processing PR # 48
Processing PR # 49
Processing PR # 50
Processing P

d:\BME\PhD\PROJ-Ericsson-CodeKnowledgeGraph\package\call_graph.py:413: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.calls = pd.concat([self.calls, new_row], ignore_index=True).reset_index(drop=True)
d:\BME\PhD\PROJ-Ericsson-CodeKnowledgeGraph\package\call_graph.py:413: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.calls = pd.concat([self.calls, new_row], ignore_index=True).reset_index(drop=True)
d:\BME\PhD\PROJ-Ericsson-CodeKnowledgeGraph\package\call_graph.py:413: FutureWarning: The 

CG build completed. Embedding CG nodes...


Batches: 100%|██████████| 162/162 [00:02<00:00, 55.36it/s] 


CG nodes embedded. Creating subgraphs for each function...


...

Subgraphs created. Embedding subgraph nodes...


100%|██████████| 1553/1553 [11:43<00:00,  2.21it/s]


Subgraph nodes embedded. Filtering graph nodes...
Graph nodes filtered. Creating hierarchical edges...
Hierarchical graph building successful.


d:\BME\PhD\PROJ-Ericsson-CodeKnowledgeGraph\package\knowledge_graph.py:451: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  temp = nodes.loc[nodes['function_location'].str.contains(filepath, na=False, case=False)]


Number of clusters: 50 with silhouette score: 0.028043834279267748


In [4]:
repograph.keys()

dict_keys(['cg_nodes', 'cg_edges', 'sg_nodes', 'sg_edges', 'hier_1', 'hier_2', 'imports', 'imp_edges', 'issues', 'prs', 'pr_edges', 'issue_to_pr_edges', 'issue_to_pr_function_edges', 'artifacts', 'actions', 'cluster_edges'])

## Save and load file

In [5]:
with open('./_/sklearn.pkl', 'wb') as f:
    pickle.dump(repograph, f)


In [ ]:
with open('_/rg.pkl', 'rb') as f:
    my_dict = pickle.load(f)

## Visualize graph

In [6]:
kgb.visualize_graph(repograph, show_subgraph_nodes=True)

Filtered sklearn graph. visualization saved to ./filtered_graph.html
